In [ ]:
# 处理cisco iosv l2 镜像telnet连接设备总是不能检测到配置模式的问题
# 正常应该是 sw-a1(config)#， 实际检测到sw-a1(config)， 缺少#号
from netmiko import Netmiko

device_params = {
    'device_type': 'cisco_ios_telnet',
    'host': '127.0.0.1',
    'port': 5000,
    'username': '',   
    'password': '',
    'secret': '',
    'timeout': 120,
}

command = [
    'interface loopback 203',
    'description netmiko test clean',
    'ip address 200.200.200.203 255.255.255.255'
]

net_connect = Netmiko(**device_params)
net_connect.enable()

try:
    print(1)
    print(net_connect.send_config_set(command))
except ValueError as e:
    if "Failed to enter configuration mode" in str(e):
        print(2)
        print(net_connect.send_config_set(command))
    else:
        print(f"other error: {e}")

net_connect.disconnect()

In [50]:
# check tool_input
import json

# example tool_intpu
input = json.dumps([
            {
                "device_name": "CiscoIOSv-1",
                "config_commands": [
                    "interface Loopback1110",
                    "ip address 201.201.201.201 255.255.255.255",
                    "description CONFIG_BY_TOOL"
                ]
            },
            {
                "device_name": "CiscoIOSvL2-1", 
                "config_commands": [
                    "interface Loopback1110",
                    "ip address 202.202.202.202 255.255.255.255",
                    "description CONFIG_BY_TOOL"
                ]
            }
        ]
)
def validate_tool_input(tool_input):
    try:
        device_configs_list = json.loads(tool_input)
        if not isinstance(device_configs_list, list):
            return [{"error": "Input must be a JSON array of device configuration objects"}]
    except json.JSONDecodeError as e:
        logger.error("Invalid JSON input: %s", e)
        return [{"error": f"Invalid JSON input: {e}"}]
    
    return device_configs_list

device_config_list = validate_tool_input(input)
print(device_config_list)

[{'device_name': 'CiscoIOSv-1', 'config_commands': ['interface Loopback1110', 'ip address 201.201.201.201 255.255.255.255', 'description CONFIG_BY_TOOL']}, {'device_name': 'CiscoIOSvL2-1', 'config_commands': ['interface Loopback1110', 'ip address 202.202.202.202 255.255.255.255', 'description CONFIG_BY_TOOL']}]


In [51]:
# tool_input json arrary to dict

def configs_map(device_config_list):
    device_configs_map = {}
    for device_config in device_config_list:
        device_name = device_config["device_name"]
        config_commands = device_config["config_commands"]
        device_configs_map[device_name] = config_commands

    return device_configs_map

device_configs_map = configs_map(device_config_list)
print(device_configs_map)


{'CiscoIOSv-1': ['interface Loopback1110', 'ip address 201.201.201.201 255.255.255.255', 'description CONFIG_BY_TOOL'], 'CiscoIOSvL2-1': ['interface Loopback1110', 'ip address 202.202.202.202 255.255.255.255', 'description CONFIG_BY_TOOL']}


In [52]:
# get gns3 device port

from public_model import get_device_ports_from_topology

def prepare_device_hosts_data(device_config_list):
    """
    """
    # 提取设备名列表
    device_names = [device_config["device_name"] for device_config in device_config_list]
    
    # 获取设备端口信息
    hosts_data = get_device_ports_from_topology(device_names)
    
    if not hosts_data:
        raise ValueError("Failed to get device information from topology or no valid devices found")
    
    # 检查是否有设备缺失
    missing_devices = set(device_names) - set(hosts_data.keys())
    if missing_devices:
        logger.warning("Some devices not found in topology: %s", missing_devices)
    
    return hosts_data


hosts_data = prepare_device_hosts_data(device_config_list)
print(hosts_data)


{'CiscoIOSv-1': {'port': 5008, 'groups': ['cisco_IOSv_telnet']}, 'CiscoIOSvL2-1': {'port': 5000, 'groups': ['cisco_IOSv_telnet']}}


In [53]:
from nornir import InitNornir
import os
# Nornir configuration groups
groups_data = {
    "cisco_IOSv_telnet": {
        "platform": "cisco_ios",
        "hostname": os.getenv("GNS3_SERVER_HOST"),
        "timeout": 120,
        "username": os.getenv("GNS3_SERVER_USERNAME"),
        "password": os.getenv("GNS3_SERVER_PASSWORD"),
        "connection_options": {
            "netmiko": {
                "extras": {
                    "device_type": "cisco_ios_telnet",
                    "fast_cli": False
                }
            }
        }
    }
}

defaults = {
    "data": {
        "location": "gns3"
    }
}

def initialize_nornir(hosts_data):
    """
    """
    try:
        return InitNornir(
            inventory={
                "plugin": "DictInventory",
                "options": {
                    "hosts": hosts_data,
                    "groups": groups_data,
                    "defaults": defaults,
                },
            },
            runner={
                "plugin": "threaded",
                "options": {
                    "num_workers": 10
                },
            },
            logging={
                "enabled": False
            },
        )
    except Exception as e:
        logger.error("Failed to initialize Nornir: %s", e)
        raise ValueError(f"Failed to initialize Nornir: {e}")

dynamic_nr = initialize_nornir(hosts_data)
print("Nornir inventory hosts:", list(dynamic_nr.inventory.hosts.keys()))
print("Inventory details:")
for host_name, host_obj in dynamic_nr.inventory.hosts.items():
    print(f"  {host_name}: {host_obj.data}")

# 检查完整的 host 对象信息
for host_name, host_obj in dynamic_nr.inventory.hosts.items():
    print(f"\n{host_name}:")
    print(f"  hostname: {host_obj.hostname}")
    print(f"  port: {host_obj.port}")
    print(f"  groups: {host_obj.groups}")
    print(f"  data: {host_obj.data}")
    print(f"  connection_options: {host_obj.connection_options}")

def simple_test_task(task):
    return Result(host=task.host, result=f"Test successful for {task.host.name}")

test_result = dynamic_nr.run(task=simple_test_task)
print("Simple test result:", test_result)

Nornir inventory hosts: ['CiscoIOSv-1', 'CiscoIOSvL2-1']
Inventory details:
  CiscoIOSv-1: {}
  CiscoIOSvL2-1: {}

CiscoIOSv-1:
  hostname: 127.0.0.1
  port: 5008
  groups: [Group: cisco_IOSv_telnet]
  data: {}
  connection_options: {}

CiscoIOSvL2-1:
  hostname: 127.0.0.1
  port: 5000
  groups: [Group: cisco_IOSv_telnet]
  data: {}
  connection_options: {}
Simple test result: AggregatedResult (simple_test_task): {'CiscoIOSv-1': MultiResult: [Result: "simple_test_task"], 'CiscoIOSvL2-1': MultiResult: [Result: "simple_test_task"]}


In [54]:
from nornir.core.task import Task, Result
from nornir_netmiko.tasks import netmiko_send_config

def run_all_device_configs_with_single_retry(task: Task) -> Result:
    """
    执行配置命令，支持单次重试机制，包含特权模式进入
    """
    device_name = task.host.name
    config_commands = device_configs_map.get(device_name, [])
    
    if not config_commands:
        return Result(host=task.host, result="No configuration commands to execute")
    
    try:
        result = task.run(
            task=netmiko_send_config,
            config_commands=config_commands
        )
        return Result(host=task.host, result=result.result)

    except Exception as e:
        if "netmiko_send_config (failed)" in str(e):
            result = task.run(
                task=netmiko_send_config,
                config_commands=config_commands
            )
            return Result(host=task.host, result=result.result)
        else:
            print(f"333333333{e}")
            return Result(
                host=task.host,
                result=f"Configuration failed (Unhandled Exception): {str(e)}",
                failed=True
                )



In [55]:

from pprint import pprint

def process_task_results(device_configs_list, hosts_data, task_result):
    """
    处理任务执行结果，为每个设备生成结果报告
    
    Args:
        device_configs_list: 原始设备配置列表
        hosts_data: 设备主机数据
        task_result: Nornir任务执行结果
    
    Returns:
        List[Dict]: 每个设备的结果列表
    """
    results = []
    
    for device_config in device_configs_list:
        device_name = device_config["device_name"]
        config_commands = device_config["config_commands"]
        
        # 检查设备是否在拓扑中
        if device_name not in hosts_data:
            device_result = {
                "device_name": device_name,
                "status": "failed",
                "error": f"Device '{device_name}' not found in topology or missing console_port"
            }
            results.append(device_result)
            continue
        
        # 检查设备是否有执行结果
        if device_name not in task_result:
            device_result = {
                "device_name": device_name,
                "status": "failed", 
                "error": f"Device '{device_name}' not found in task results"
            }
            results.append(device_result)
            continue
        
        # 处理执行结果
        multi_result = task_result[device_name]
        device_result = {"device_name": device_name}
        
        if multi_result[0].failed:
            # 执行失败
            device_result["status"] = "failed"
            device_result["error"] = f"Configuration execution failed: {multi_result[0].result}"
            device_result["output"] = multi_result[0].result
        else:
            # 执行成功
            device_result["status"] = "success"
            device_result["output"] = multi_result[0].result
            device_result["config_commands"] = config_commands
        
        results.append(device_result)
    
    return results

task_result = dynamic_nr.run(
    task=run_all_device_configs_with_single_retry
)

output =  process_task_results(
    device_config_list,
    hosts_data,
    task_result
)
pprint(output)

Host 'CiscoIOSvL2-1': task 'netmiko_send_config' failed with traceback:
Traceback (most recent call last):
  File "/home/yueguobin/gns3-copilot/venv/lib/python3.11/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yueguobin/gns3-copilot/venv/lib/python3.11/site-packages/nornir_netmiko/tasks/netmiko_send_config.py", line 38, in netmiko_send_config
    result = net_connect.send_config_set(config_commands=config_commands, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yueguobin/gns3-copilot/venv/lib/python3.11/site-packages/netmiko/base_connection.py", line 111, in wrapper_decorator
    return_val = func(self, *args, **kwargs)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yueguobin/gns3-copilot/venv/lib/python3.11/site-packages/netmiko/base_connection.py", line 2305, in send_config_set
    output += self.config_mode()
   

[{'config_commands': ['interface Loopback1110',
                      'ip address 201.201.201.201 255.255.255.255',
                      'description CONFIG_BY_TOOL'],
  'device_name': 'CiscoIOSv-1',
  'output': '\n'
            'BR-1#\n'
            'BR-1#configure terminal\n'
            'Enter configuration commands, one per line.  End with CNTL/Z.\n'
            'BR-1(config)#\n'
            'BR-1(config)#\n'
            'BR-1(config)#interface Loopback1110\n'
            'BR-1(config-if)#\n'
            'BR-1(config-if)#ip address 201.201.201.201 255.255.255.255\n'
            'BR-1(config-if)#\n'
            'BR-1(config-if)#description CONFIG_BY_TOOL\n'
            'BR-1(config-\n'
            'BR-1(config-if)#\n'
            'BR-1(config-if)#\n'
            'BR-1(config-if)#end\n'
            'BR-1#',
  'status': 'success'},
 {'config_commands': ['interface Loopback1110',
                      'ip address 202.202.202.202 255.255.255.255',
                      'description CON